#### 2.
В облачном колл-центре Тинькофф операторы продаж получают деньги за проданные продукты клиенту, время диалога с клиентом. За продажу одних и тех же продуктов из разных баз существует своя стоимость успешного результата продажи, а за продажу дополнительных продуктов оператор получает еще и дополнительное вознаграждение. Необходимо автоматизировать выплаты операторам, для этого создается новый сервис для расчета заработной платы за расчетный период времени. Предполагается после каждого разговора оператора с клиентом отправлять бизнес-данные в новый сервис для последующего расчета заработной платы (в случае разрыва сети и перезвона - новая отправка). Необходимо составить JSON для отправки сообщения в новый сервис.

1) Формируем json-схему для отправки сообщения в новый сервис.

2) Формируем json-объекты на основании полученных данных колл-центра.

3) Если объект прошел валидацию json-схемы, отправляем запрос серверу, в одном из параметров запроса передаем json-объект. В противном случае, формируем файл с логами.

Логика расчета заработной платы за расчетный период времени в новом сервисе:

1. Одновременно фильтруем все записи:
       1.1. по началу и концу расчетного периода, используя данные полей "start_datetime_transaction" и/или "end_datetime_transaction",
       1.2. по сотруднику, используя данные объекта "operator".
       
2. По каждой отфильтрованной строке выполняем следующий алгоритм:
       2.1. рассчитываем время диалога с клиентом на основании полей "start_datetime_transaction" и "end_datetime_transaction",
       2.2. с учетом параметра "add_prod" устанавливаем дополнительные продукты /основной продукт,
       2.3. на основании данных поля "dbo_id" для каждого продукта ("id_prod") устанавливаем стоимость успешного результата продажи,
       2.4. по каждому дополнительному продукту учитываем дополнительное вознаграждение,
       2.5. на основании данных выше рассчитываем суммарную зарплату сотрудника колл-центра за один телефонный звонок.
       
3. Суммируем результаты, полученные по каждой отфильтрованной строке.   

In [83]:
import requests
from jsonschema import validate, Draft7Validator

In [84]:
schema = {
    "title": "transaction",
    "type" : "object",
    "required": [ "id_transaction", "operator", "start_datetime_transaction", "end_datetime_transaction", "product_dict" ],
    "properties" : {
        "id_transaction": {
                    "description": "Unique operator call identifier",
                    "type": "string"
        },
        "start_datetime_transaction": {
                    "description": "Start date and time of operator call",
                    "type" : "string"
        },
        "end_datetime_transaction": {
                    "description": "End date and time of operator call",
                    "type" : "string"
        },
        "operator": {
            "type": "object",
            "required": ["id","first_name","last_name"],
            "properties": {
                    "id": { 
                        "description": "Operator unique identifier",
                        "type": "string"
                    },
                    "first_name": { 
                        "description": "Operator first_name",
                        "type": "string"
                    },
                    "last_name": { 
                        "description": "Operator last_name",
                        "type": "string"
                    },
                    "middle_name": { 
                        "description": "Operator middle_name",
                        "type": "string"
                    }
            }
        },
        "product_dict": {
             "type": "array",
             "minItems": 1,
             "items":{
                 "type": "object",
                 "required": [ "id_prod", "dbo_id", "add_prod"],
                 "properties": {
                    "id_prod": {
                        "description": "Item_id",
                        "type": "string"
                    },
                    "dbo_id": {
                        "description": "SQL_id",
                        "type": "string"
                    },
                    "add_prod": {
                        "description": "Additional product or not",
                        "type": "boolean"                        
                    }
                 }
             } 
        }
    }
}

In [88]:
# для проверки json-объекты сформированы вручную
json_list = []

json_transaction_1={"id_transaction": "Rec_01233", 
                    "start_datetime_transaction": "2021-01-09T12:54:03.433793",
                    "operator": "Maria"}
json_list.append(json_transaction_1)

json_transaction_2 = {"id_transaction": "Rec_01234", 
                    "start_datetime_transaction": "2021-01-09T12:54:03.433793",
                    "end_datetime_transaction": "2021-01-09T13:54:03.433793",
                    "operator": {"id": "123", "first_name" : "Andrey", "last_name" : "Ivanov"},
                    "product_dict" : [
                        { "id_prod" : "id_123", "dbo_id" : "server/dbo_name_1", "add_prod" : True},
                        { "id_prod" : "id_127", "dbo_id" : "server/dbo_name_1", "add_prod" : False},
                        { "id_prod" : "id_123", "dbo_id" : "server/dbo_name_2", "add_prod" : True}
                    ]}
json_list.append(json_transaction_2)

In [90]:
v = Draft7Validator(schema)
for i,item in enumerate(json_list):
    errors = sorted(v.iter_errors(item), key=lambda e: e.path)
    print(f'{item["id_transaction"]}\n')
    for error in errors:
        print(f'{error.message}\n')
    if errors == []:
        print("post_request")
        #resp = requests.post(url="exist_server_url", params=json_transaction_1)
    else:
        with open('log.txt', 'w', encoding='utf8') as f:
            f.write(f'{item["id_transaction"]}\n')
            for error in errors:
                f.write(f'{error.message}\n')

Rec_01233

'end_datetime_transaction' is a required property

'product_dict' is a required property

'Maria' is not of type 'object'

Rec_01234

post_request
